In [1]:
import os
from pathlib import Path
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
def load_data(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [3]:
data_root = r"C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc"

In [4]:
file_path = Path(data_root, 'data/processed/facial-1007-sequential.pkl')

In [5]:
data = load_data(file_path)

In [6]:
X,Y = data

In [7]:
X.shape, Y.shape

((34880, 30, 323), (34880, 5))

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import MultiHeadAttention, Embedding

In [10]:
# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [11]:
# Define a Transformer block as a layer
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build the model
model = Sequential([
    TransformerBlock(embed_dim=323, num_heads=8, ff_dim=512),
    Dense(5)  # Linear activation for regression
])


In [12]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mse'])

In [13]:
# Summary of the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ transformer_block (TransformerBlock) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
X_train.shape, Y_train.shape

((27904, 30, 323), (27904, 5))

In [15]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_data=(X_val, Y_val))

Epoch 1/10


ValueError: Layers added to a Sequential model can only have a single positional argument, the input tensor. Layer TransformerBlock has multiple positional arguments: [<Parameter "inputs">, <Parameter "training">]

In [20]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [21]:
# Define a Transformer block as a layer
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build the model using the Functional API
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = TransformerBlock(embed_dim=323, num_heads=8, ff_dim=512)(inputs)
outputs = Dense(5)(x)  # Linear activation for regression
model = Model(inputs=inputs, outputs=outputs)

In [22]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mse'])

In [23]:

# Summary of the model
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 30, 323)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_2                  │ (None, 30, 323)             │       3,679,482 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 30, 5)               │           1,620 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,681,102 (14.04 MB)

 Trainable params: 3,681,102 (14.04 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Train the model
history = model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_data=(X_val, Y_val))

Epoch 1/10


ValueError: Dimensions must be equal, but are 64 and 30 for '{{node compile_loss/mean_squared_error/sub}} = Sub[T=DT_FLOAT](data_1, functional_3_1/dense_8_1/Add)' with input shapes: [64,5], [64,30,5].

In [26]:
from tensorflow.keras.layers import GlobalAveragePooling1D

In [27]:
# Define a Transformer block as a layer
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build the model using the Functional API
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = TransformerBlock(embed_dim=323, num_heads=8, ff_dim=512)(inputs)
x = GlobalAveragePooling1D()(x)  # Reduce the sequence dimension
outputs = Dense(5)(x)  # Linear activation for regression
model = Model(inputs=inputs, outputs=outputs)

In [28]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mse'])

In [29]:
# Summary of the model
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 30, 323)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_4                  │ (None, 30, 323)             │       3,679,482 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 323)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 5)                   │           1,620 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,681,102 (14.04 MB)

 Trainable params: 3,681,102 (14.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_data=(X_val, Y_val))

Epoch 1/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 106s 238ms/step - loss: 0.7228 - mse: 0.7228 - val_loss: 0.0026 - val_mse: 0.0026
Epoch 2/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 104s 239ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.0027 - val_mse: 0.0027
Epoch 3/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 103s 237ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0028 - val_mse: 0.0028
Epoch 4/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 103s 237ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.0018 - val_mse: 0.0018
Epoch 5/10
295/436 ━━━━━━━━━━━━━━━━━━━━ 32s 229ms/step - loss: 0.0024 - mse: 0.0024